# Algoritmo Genético para treinamento da RNA

Importe os pacotes necessários

In [16]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from imutils import paths
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
from keras import optimizers
import numpy as np
from numpy import array, dtype
import argparse
import random
import pickle
import cv2
import os
import livelossplot
import collections
import asyncio
import sys


# Estrutura do Algoritmo Genético

## Operações para definir os pesos da RNA

In [17]:
def inserePesosConv2d(lista, j, k, h):
	qtdPesos = 32
	g = 0
	while(g < qtdPesos):
		peso = random.uniform(-1.0, 1.0)
		lista[j][k][h].append(peso)
		g += 1
	return lista

def montaConv2d_1():
	camada = list()
	lista = list() ## no primeiro caso deve ter 3 posições, no segundo caso deve ter 32 posições

	descendente = 2 ## quantidade de elementos na camada

	i = 0
	while(i < descendente):
		lista = []
		if( i == 0):
			tamanhoSubCamada = 3
			j = 0
			while(j < tamanhoSubCamada):
				subLista = list()
				lista.append(subLista)

				tamanhoSubSubCamada = 3
				k = 0
				while(k < tamanhoSubSubCamada):
					subsubLista = list()
					lista[j].append(subsubLista)

					tamanhoSubSubSubcamada = 3
					h = 0
					while(h < tamanhoSubSubSubcamada):
						subsubsubLista = list()
						lista[j][k].append(subsubsubLista)

						lista = inserePesosConv2d(lista, j, k, h)

						h += 1
					k += 1
				j += 1
						
		elif(i == 1):
			qtdPesos = 32
			j = 0
			while(j < qtdPesos):
				lista.append(0.0)
				j += 1
		else:
			print("Não pode ter mais que duas posições")
		
		camada.append(np.array(lista, dtype='float32'))
		i += 1
	return camada



def montaCamada(arraysNaCamada, qtdPorArray):
	camada = list()
	return camada 

def montaBatch_normalization(arraysNaCamada, qtdPorArray):
	camada = list()
	if(not arraysNaCamada == 0):
		i = 0
		lista = None
		
		while(i < arraysNaCamada):
			lista = list()
			
			pesoAtual = 0
			while(pesoAtual < qtdPorArray[i]):
				if(i == 0 or i == 3):
					numero = 1.0
				else:
					numero = 0.0

				lista.append(numero)
				pesoAtual += 1

			camada.append(np.array(lista, dtype='float32'))
			i = i + 1

	return camada 

def montaPesoNaDensa(lista):
	qtdPesos = 32768
	g = 0
	while(g < qtdPesos):
		sub = []
		peso = random.uniform(-1.0, 1.0)
		sub.append(peso)
		lista.append(sub)
		g += 1
	return lista

def montaDensa(arraysNaCamada, qtdPorArray):
	camada = list()
	if(not arraysNaCamada == 0):
		i = 0
		lista = None
		
		while(i < arraysNaCamada):
			lista = []	
			DEFAULT = 0.0
			if(i == 0):
				result = montaPesoNaDensa(lista)
			elif(i == 1):
				peso = DEFAULT
				lista.append(peso)
			else:
				print("Nao era pra cair aqui")

			i = i + 1
			camada.append(np.array(lista, dtype='float32'))
	
	return camada 

arraysNaCamada = None
qtdPorArray = None

## Inicia um Indivíduo

In [18]:
def init_individual():
    individual = []
    array0 = np.array([],dtype='float32')
    array1 = montaConv2d_1() 
    array2 = montaCamada(0, []) # é vazio
    array3 = montaBatch_normalization(4, [32,32,32,32])
    array4 = montaCamada(0, []) # é vazio
    array5 = montaCamada(0, []) # é vazio
    array6 = montaDensa(2, [32768, 1])
    array7 = montaBatch_normalization(4, [1,1,1,1])
    individual.append(array0)
    individual.append(array1)
    individual.append(array2)
    individual.append(array3)
    individual.append(array4)
    individual.append(array5)
    individual.append(array6)
    individual.append(array7)
    
    return individual


## Inicia uma nova População

In [19]:
def init_new_population(population_size):
    
    new_population = []
    
    iterator = 0
    while (iterator < population_size):
        individual = init_individual()
        new_population.append(individual)
        iterator = iterator + 1
    
    return(new_population)

## Execução da RNA

In [11]:
def execute_rna(list0, list1, list2, list3, list4, list5, list6, list7):

    dataset = 'dataset'

    EPOCHS = 3
    INIT_LR = 1e-3
    MOMENTUM = 0.8
    BS = 5
    IMAGE_DIMS = (96, 96, 3)
    modelname = 'blood.model'
    label = 'lb.pickle'

    data = []
    labels = []

    imagePaths = list(paths.list_images(dataset))

    for imagePath in imagePaths:
        # carrega a imagem, pré-processa e armazena na lista de dados
        image = cv2.imread(imagePath)
        image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
        image = img_to_array(image)
        data.append(image)

        # extrai o rótulo da classe do caminho da imagem e atualiza a lista de rótulos
        label = imagePath.split(os.path.sep)[-2]
        labels.append(label)

    data = np.array(data, dtype="float") / 255.0
    labels = np.array(labels)
    print("[INFO] data matrix: {:.2f}MB".format(
        data.nbytes / (1024 * 1000.0)))

    lb = LabelBinarizer()
    labels = lb.fit_transform(labels)

    (trainX, testX, trainY, testY) = train_test_split(data,
        labels, test_size=0.2, shuffle=False )

    aug = ImageDataGenerator()
    width=IMAGE_DIMS[1]
    height=IMAGE_DIMS[0]
    depth=IMAGE_DIMS[2]
    classes=len(lb.classes_)

    model = Sequential()
    inputShape = (height, width, depth)
    chanDim = -1

    if K.image_data_format() == "channels_first":
        inputShape = (depth, height, width)
        chanDim = 1

    # CONV => RELU => POOL
    model.add(Conv2D(32, (3, 3), padding="same",
        input_shape=inputShape, use_bias=True))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(3, 3)))

    #set of FC => RELU layers
    model.add(Flatten())
    model.add(Dense(1))
    model.add(BatchNormalization())


#     print('original')
#     print('#########################################################')
#     for layer in model.layers:
#         weights = layer.get_weights()
#     #     print(weights)

    # layer0 = layer[0].get_weights()
    #print(model.layers)
#     print(model.get_layer(Conv2D,0).get_weights())
#     print(model.get_layer(Conv2D,1).get_weights())
#     print(model.get_layer(Conv2D,2).get_weights())
#     print(model.get_layer(Conv2D,3).get_weights())
#     print(model.get_layer(Conv2D,4).get_weights())
#     print(model.get_layer(Conv2D,5).get_weights())
#     print(model.get_layer(Conv2D,6).get_weights())
#     print(model.get_layer(Conv2D,7).get_weights())

    #model.summary()    

    # Classificador
    model.add(Dense(classes))
    model.add(Activation("tanh")) #tanh <- Tangente hiperbólica
    opt = optimizers.SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS, momentum=MOMENTUM, nesterov=True)
    
    model.get_layer(Conv2D,0).set_weights(list0)
    model.get_layer(Conv2D,1).set_weights(list1)
    model.get_layer(Conv2D,2).set_weights(list2)
    model.get_layer(Conv2D,3).set_weights(list3)
    model.get_layer(Conv2D,4).set_weights(list4)
    model.get_layer(Conv2D,5).set_weights(list5)
    model.get_layer(Conv2D,6).set_weights(list6)
    model.get_layer(Conv2D,7).set_weights(list7)
    
#     print(model.get_layer(Conv2D,1).get_weights()) 
#     print(model.get_layer(Conv2D,1).get_weights())
#     print(model.get_layer(Conv2D,2).get_weights())
#     print(model.get_layer(Conv2D,3).get_weights())
#     print(model.get_layer(Conv2D,4).get_weights())
#     print(model.get_layer(Conv2D,5).get_weights())
#     print(model.get_layer(Conv2D,6).get_weights())
#     print(model.get_layer(Conv2D,7).get_weights())
    
    model.compile(loss="categorical_crossentropy", optimizer=opt,
        metrics=["accuracy"])
    plot_losses = livelossplot.PlotLossesKeras()
    H = model.fit_generator(
        aug.flow(trainX, trainY, batch_size=BS),
        validation_data=(testX, testY),
        steps_per_epoch=len(trainX) // BS,
        epochs=EPOCHS,
        #callbacks=[plot_losses],
        verbose=1
    )
    train_loss = H.history["loss"]
    val_loss = H.history["val_loss"]
    train_acc = H.history["acc"]
    val_acc = H.history["val_acc"]

    fitness = (sum(train_acc)/len(train_acc))
    return fitness

## Testa cada indivíduo da População e obtém o fitness

In [22]:
def get_fitness(population):
    #inicia a matrix 2x6 da população
    population_matrix = [[0]*2 for i in range(6)]
    
    #Obtém fitness do indivíduo 1
    population_matrix[0][0] = population[0]
    population_matrix[0][1] = execute_rna(population[0][0], population[0][1], population[0][2], population[0][3], population[0][4], population[0][5], population[0][6], population[0][7])

    
    #Obtém fitness do indivíduo 2
    population_matrix[1][0] = population[1]
    population_matrix[1][1] = execute_rna(population[1][0], population[1][1], population[1][2], population[1][3], population[1][4], population[1][5], population[1][6], population[1][7])

    #Obtém fitness do indivíduo 3
    population_matrix[2][0] = population[2]
    population_matrix[2][1] = execute_rna(population[2][0], population[2][1], population[2][2], population[2][3], population[2][4], population[2][5], population[2][6], population[2][7])

    #Obtém fitness do indivíduo 4
    population_matrix[3][0] = population[3]
    population_matrix[3][1] = execute_rna(population[3][0], population[3][1], population[3][2], population[3][3], population[3][4], population[3][5], population[3][6], population[3][7])

    #Obtém fitness do indivíduo 5
    population_matrix[4][0] = population[4]
    population_matrix[4][1] = execute_rna(population[4][0], population[4][1], population[4][2], population[4][3], population[4][4], population[4][5], population[4][6], population[4][7])

    #Obtém fitness do indivíduo 6
    population_matrix[5][0] = population[5]
    population_matrix[5][1] = execute_rna(population[5][0], population[5][1], population[5][2], population[5][3], population[5][4], population[5][5], population[5][6], population[5][7])

    return(population_matrix)

population = init_new_population(6)
population_fitness = get_fitness(population)
print(population_fitness[0][1])
print(population_fitness[1][1])
print(population_fitness[2][1])
print(population_fitness[3][1])
print(population_fitness[4][1])
print(population_fitness[5][1])

[INFO] data matrix: 4.32MB
Epoch 1/3
3/3 [==============================] - 2s 550ms/step - loss: nan - acc: 0.1455 - val_loss: nan - val_acc: 0.0000e+00
Epoch 2/3
3/3 [==============================] - 0s 34ms/step - loss: nan - acc: 0.2667 - val_loss: nan - val_acc: 0.0000e+00
Epoch 3/3
3/3 [==============================] - 0s 29ms/step - loss: nan - acc: 0.3636 - val_loss: nan - val_acc: 0.0000e+00
[INFO] data matrix: 4.32MB
Epoch 1/3
3/3 [==============================] - 2s 588ms/step - loss: 7.4737 - acc: 0.2000 - val_loss: 4.4456 - val_acc: 0.5000
Epoch 2/3
3/3 [==============================] - 0s 31ms/step - loss: 4.1949 - acc: 0.0727 - val_loss: 4.4593 - val_acc: 0.5000
Epoch 3/3
3/3 [==============================] - 0s 28ms/step - loss: 6.0629 - acc: 0.3714 - val_loss: 4.4836 - val_acc: 0.5000
[INFO] data matrix: 4.32MB
Epoch 1/3
3/3 [==============================] - 2s 648ms/step - loss: nan - acc: 0.3636 - val_loss: nan - val_acc: 0.0000e+00
Epoch 2/3
3/3 [=============

In [5]:
def select_best_individuals(population):

    list = [2,5,342,12,234,1234,23]
    list.sort()
    print(list)

population = init_new_population(6)
population_fitness = get_fitness(population)
    
select_best_individuals()

[2, 5, 12, 23, 234, 342, 1234]


In [6]:
def crossover_individuals():
    """
    Slices both dna1 and dna2 into two parts at a random index within their
    length and merges them. Both keep their initial sublist up to the crossover
    index, but their ends are swapped.
    """
    list1 = [1,2,3,4]
    list2 = [11,12,13,14]
    pos=int(random.random()*len(list1))
    print(pos)
    print(list1, list2)
    print(list1[:pos]+list2[pos:], list2[:pos]+list1[pos:])
    
crossover_individuals()

3
[1, 2, 3, 4] [11, 12, 13, 14]
[1, 2, 3, 14] [11, 12, 13, 4]


In [7]:
def random_float():
    """
    Return a random number between -1 and 1
    """
    return random.uniform(-1, 1)

In [8]:

def mutate():
    """
    For each gene in the DNA, there is a 1/mutation_chance chance that it will be
    switched out with a random character. This ensures diversity in the
    population, and ensures that is difficult to get stuck in local minima.
    """
    mutation_chance = 100
    index = 0
    dna_out=[]
    list1 = [1,2,3,4,5,6,7,8,9,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]

    for i in list1:
        if int(random.random()*mutation_chance) == 1:
            dna_out.insert(index, random_float())
        else:
            dna_out.insert(index, i)
        index+1

    print(list(reversed(dna_out)))

mutate()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
